<a href="https://colab.research.google.com/github/gpugems/llm_demo/blob/main/class_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install python-dotenv openai qdrant-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.2 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.29.2 which is incompatible.


In [41]:
import os
import openai
import pandas as pd

from openai import OpenAI
from google.colab import userdata

from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')



In [36]:

df = pd.read_csv('sample_data/top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

700

In [42]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [43]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [44]:
# Create collection to store books
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

<ipython-input-44-5de2cf0f8439>:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [46]:
# vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [47]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode("A wine from Mendoza Argentina").tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Kistler Vineyards Kistler Vineyard Pinot Noir 1998', 'region': 'Sonoma Valley, Sonoma County, California', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'A concentrated wine, with impressive energy to its red and black fruit flavors and subtle earth tones.'} score: 0.573888326540054
{'name': 'Catena Zapata Adrianna Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 97.0, 'notes': '"The single-vineyard 2004 Malbec Adrianna Vineyard from the Gualtallary district is inky purple with aromas of wood smoke, pencil lead, game, black cherry, and blackberry liqueur. Opulent, full-flavored, yet remarkably light on its feet, this medium to full-bodied Malbec is all about pleasure. It will certainly evolve for a decade but is hard to resist now. It is a fine test of one\'s ability to defer immediate gratification. When all is said and done, Catena Zapata is the Argentina winery of reference – the standard of excellence for comparing all others. The brilliant

In [50]:
search_results = [hit.payload for hit in hits]

In [51]:
Client = OpenAI()



completion = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": str(search_results)}
    ],
)
print(completion.choices[0].message.content)



One exceptional Malbec from Argentina that stands out is the **Catena Zapata Adrianna Vineyard Malbec 2004**. This wine is renowned for its deep, inky purple color and remarkable aromas of wood smoke, pencil lead, game, black cherry, and blackberry liqueur. It offers an opulent and full-flavored experience, yet remains remarkably light on the palate. While it possesses great aging potential, it's also tempting to enjoy it now for its vibrant and complex profile. Produced by Catena Zapata, a benchmark winery in Mendoza, this Malbec exemplifies the excellence and innovation that have positioned Argentina as a leader in the global wine scene.
